# COIN-OR

In [1]:
# Datenzugriff
from configs.config import get_path
import json

# Utils
import src.utils.presenter as show
from src.utils.production_process import convert
from src.utils.production_process import initialize as init
from src.utils.analize import check_constrains as check

# Solver Model
from src.models.lp import makespan

# conda install -c conda-forge highs

In [2]:
max_time = 60*10 # 10 min

In [3]:
# Datei laden
basic_data_path = get_path("data", "basic")
file_path = basic_data_path / "jobshop_instances.json"

with open(file_path, "r", encoding="utf-8") as f:
    jobshop_instances = json.load(f)

instance =  jobshop_instances["instance ft10"]
df_instance = convert.jssp_dict_to_df(instance)

df_production_orders = init.production_orders(df_instance)
df_production_orders

,Job,Production_Plan_ID,Operation,Machine,Processing Time
0,0,0,0,M00,29
1,0,0,1,M01,78
2,0,0,2,M02,9
3,0,0,3,M03,36
4,0,0,4,M04,49
...,...,...,...,...,...
95,9,9,5,M09,76
96,9,9,6,M05,47
97,9,9,7,M03,52
98,9,9,8,M04,90


### a) CBC - COIN-OR Branch and Cut

In [ ]:
# Scheduling
df_schedule = makespan.solve_jssp(df_production_orders, solver= "CBC", var_cat="Integer", msg=False, time_limit=max_time)
df_schedule

BigM: 5200


In [ ]:
show.plot_gantt(df_schedule, perspective="Job")

In [ ]:
show.plot_gantt(df_schedule, perspective="Machine")
check.is_machine_conflict_free(df_schedule)
check.is_operation_sequence_correct(df_schedule)
check.is_job_timing_correct(df_schedule)

### b) HiGHS mit PuLP

> high performance serial and parallel solver

In [ ]:
# Scheduling
df_schedule = makespan.solve_jssp(df_production_orders, solver= "HiGHS", var_cat="Integer", msg=False, time_limit=max_time)
df_schedule

In [ ]:
show.plot_gantt(df_schedule, perspective="Job")

In [ ]:
show.plot_gantt(df_schedule, perspective="Machine")
check.is_machine_conflict_free(df_schedule)
check.is_operation_sequence_correct(df_schedule)
check.is_job_timing_correct(df_schedule)